# Creating a metabolic hierarchical tree from the Metacyc database

## Background

From the [EcoCyc database](http://ecocyc.org/PToolsWebsiteHowto.shtml#node_sec_8): 

Organization of the Cellular Overview: Within the cytoplasmic membrane, the small-molecule metabolism of the organism is depicted in several regions. **The glycolysis and the TCA cycle pathways, if present, will be placed in the middle of the diagram** to separate **predominately catabolic pathways on the right** from pathways of **anabolism and intermediary metabolism on the left**. The existence of anaplerotic pathways prevents rigid classification. The majority of pathways operate in the downward direction.

Here we will create a network representation of the Metacyc database. Nodes will be pathways and edges between nodes  are compounds linking the pathways. In order to emulate the Cellular Overview described above three networks will be generated, representing **Biosynthesis**, **Core** and **Degradation** pathways. These networks will then be composed together.

The resulting graph can be used to overlay pathway abundance data from sequencing experiments.

## Method

In [ ]:
import networkx as nx, matplotlib.pyplot as plt, pandas as pd, pickle, matplotlib as mpl

from glob import glob
import plotly.plotly as py
from plotly.graph_objs import *
import numpy as np
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

%config InlineBackend.figure_format = 'svg'
%matplotlib inline
plt.style.use('ggplot')

### Read the metacyc data

Select graph table and Enzyme/Pathway table for Metacyc. Choose **GitHub** to read from the [parse-db](https://github.com/johnne/parse-db/metacyc) repository.

In [ ]:
def read_metacyc(f,g):
    if g=="GitHub": g = "https://github.com/johnne/parse-db/raw/master/metacyc/db_data/metacyc.pathway_graph.tab"
    if f=="GitHub": f = "https://github.com/johnne/parse-db/raw/master/metacyc/db_data/metacyc.ec2pathcats.tab"
    global metacyc_graph
    global metacyc_df
    metacyc_graph = pd.read_csv(g, header=0, sep="\t", index_col=0)
    metacyc_df = pd.read_csv(f, header=0, sep="\t", index_col=0)
    
g_sel = widgets.Select(options=glob("datadir/*")+["GitHub"], description="Graph table", value="GitHub")
ec_sel = widgets.Select(options=glob("datadir/*")+["GitHub"], description="EC2Path table", value="GitHub")
interact(read_metacyc,f=ec_sel,g=g_sel, __manual=True)

### Select pathways of interest (optional)

In [ ]:
def select_pathways(f,df):
    p_start = len(set(df.Pathway))
    p = pd.read_csv(f,index_col=0,header=None)
    df = df.loc[df.Pathway.isin(p.index)]
    metacyc_df = df.copy(deep=True)
    p_end = len(set(metacyc_df.Pathway))
    print("Went from",p_start,"to",p_end,"pathways.")
path_sel = widgets.Select(options=glob("datadir/*"),description="Pathways")
interact(select_pathways,f=path_sel, df=fixed(metacyc_df), __manual=True)

### Make subsets of pathways

As seen in the Venn diagram below (based on Metacyc v. 20.1) there is some overlap in terms of categorization of pathways to higher levels in the hierarchy.

<p align="left">
    <img src="https://github.com/johnne/parse-db/raw/master/metacyc/images/metacyc_venn.png" width="300" height="300"/>
</p>

To make things easy for us, pathways that overlap in some way between the three different categories will first be put into the 'Core' (Generation of Precursor Metabolites and Energy) category, then the 'Degradation' (Degradation/Utilization/Assimilation).

In [ ]:
core = ["Generation of Precursor Metabolites and Energy"]
syn = ["Biosynthesis"]
deg = ["Degradation/Utilization/Assimilation"]

core_pwys = list(set(metacyc_df.loc[metacyc_df.Category1.isin(core),"Pathway"]))
deg_pwys = list(set(metacyc_df.loc[metacyc_df.Category1.isin(deg),"Pathway"]))
deg_pwys = list(set(deg_pwys).difference(set(core_pwys)))
syn_pwys = list(set(metacyc_df.loc[metacyc_df.Category1.isin(syn),"Pathway"]))
syn_pwys = list(set(syn_pwys).difference(set(core_pwys+deg_pwys)))

In [ ]:
def add_subset(root,pwys,metacyc_df,col='b'):
    dg = nx.DiGraph()
    df = metacyc_df.loc[(metacyc_df.Category1==root) & (metacyc_df.Pathway.isin(pwys))]
    df = df.groupby("Pathway").first()
    df = df.loc[:,["Category1","Category2","Category3","Category4","Category5","Category6","Category7","Parent","Name"]]
    ## Add directed paths for all pathways starting at the root
    for p in pwys:
        r = df.loc[p]
        previous = root
        for i in range(2,8):
            c = "Category"+str(i)
            current = r[c]
            ## If missing value for this level, use previous level name
            if current!=current: current = previous
            ## Add suffix to indicate level in the hierarchy
            current = current.split("|")[0]+"|"+str(i)
            ## Add edge to the current level
            dg.add_edge(previous,current)
            ## Assign color value
            dg.node[current]['col'] = col 
            dg.node[previous]['col'] = col
            previous = current
        i = 8
        current = r["Parent"]
        if current!=current: current = r["Parent"]
        current = current.split("|")[0]+"|"+str(i)
        ## Add the parent node
        dg.add_edge(previous,current)
        dg.node[current]['col'] = col
        dg.node[previous]['col'] = col
        ## Add final pathway node
        dg.add_edge(current,p)
        dg.node[p]['col'] = col
        name = r["Name"]
        dg.node[p]['Name'] = name
    return dg

dg_c = add_subset(core[0],core_pwys,metacyc_df,'r')
dg_s = add_subset(syn[0],syn_pwys,metacyc_df,'b')
dg_d = add_subset(deg[0],deg_pwys,metacyc_df,'g')
DG = nx.compose_all([dg_d,dg_s,dg_c])

### Create the graphs

#### Functions to order nodes and assign node attributes

In [ ]:
def assign_node_weights(g,d):
    for n in g.nodes():
        try: w = d[n]
        except KeyError: w = 0
        try: g.node[n]['weight'] = w
        except KeyError: g.node[n] = {'weight': w}
    return g

def assign_node_pos(g,pos):
    for n in g.nodes(): g.node[n]['pos'] = pos[n]
    return g

def shift_nodes(g,nodes,x,y):
    for n in nodes:
        xn,yn = g.node[n]['pos']
        xn+=x
        yn+=y
        g.node[n]['pos'] = (xn,yn)
    return g

#### Final touches

Here the subgraphs are composed together and node the number of connections added as 'node weights'. Finally, edges are added connecting pathways between subgraphs.

In [ ]:
## Count edges for pathways
edge_counts = metacyc_graph.groupby("Node1").count().ix[:,0]
edge_counts = edge_counts.sort_values(ascending=False)

## Count descendants for nodes
descendants = {}
for node in DG: descendants[node] = len(nx.descendants(DG,node))

## Assign node weights
DG = assign_node_weights(DG,descendants)

## Assign positions
pos=nx.nx_pydot.graphviz_layout(DG,prog='dot')
DG = assign_node_pos(DG,pos)

In [ ]:
## Shift positions for core and degradation
DG = shift_nodes(DG,dg_c.nodes(),-78010.0,0)
DG = shift_nodes(DG,dg_d.nodes(),78010.0,0)

## Visualize the resulting graph

### Using plotly

In [ ]:
def plotly_graph(G, df, f,text):
    ## Add colorbar for nodes, using log scale
    ## Use log scale for colorbar
    tickvals = np.log([1,101,201,501,1001,1501,2501])
    ticktext = ["0","100","200","500","1000","1500","2500"]
    node_colorbar=dict(thickness=15,len=0.5,title='Descendants',xanchor='left',
                       titleside='right',tickmode='array',ticktext=ticktext, tickvals=tickvals)    

    ## Add edge and node traces
    edge_trace = Scatter(x=[],y=[],text=[],line=Line(width=0.1,color='#888'),hoverinfo='text',mode='lines')
    node_trace = Scatter(x=[],y=[],text=[],mode='markers',hoverinfo='text',marker=Marker(showscale=True,colorscale='RdBu',
        reversescale=False,color=[],size=[],colorbar=node_colorbar,line=dict(width=0.5)))

    ## Create figure layout
    fig_layout = Layout(title="Metacyc tree",titlefont=dict(size=12),showlegend=False,width=1200,height=500,
                hovermode='closest',margin=dict(b=20,l=5,r=5,t=20),
                annotations=[dict(showarrow=False,text="Biosynthesis",x=68685, y=620),
                            dict(showarrow=False,text="Core metabolism",x=222800, y=620),
                            dict(showarrow=False,text="Degradation",x=300810, y=620),
                            dict(showarrow=False,x=200000,y=0,
                                 text="Jupyter notebook on GitHub: <a href='https://github.com/johnne/biovisualize/blob/master/metacyc/create_metacyc_tree.ipynb'> create_metacyc_tree.ipynb</a>")],
                xaxis=XAxis(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=YAxis(showgrid=False, zeroline=False, showticklabels=False))

    ## Add edges to edge trace
    for edge in G.edges():
        x0, y0 = G.node[edge[0]]['pos']
        x1, y1 = G.node[edge[1]]['pos']
        edge_trace['x'] += [x0, x1, None]
        edge_trace['y'] += [y0, y1, None]

    ## Add nodes to node trace
    for n in G.nodes():
        x, y = G.node[n]['pos']
        node_trace['x'].append(x)
        node_trace['y'].append(y)
        node_trace['marker']['color'].append(np.log(G.node[n]['weight']+1))
        try: name = list(set(df.loc[df.Pathway==n,"Name"]))[0]
        except IndexError: name = n
        node_trace['marker']['size'].append(6)
        node_info = name+" (#"+str(G.node[n]['weight'])+")"
        node_trace['text'].append(node_info)

    fig = Figure(data=Data([edge_trace, node_trace]), layout=fig_layout)
    return py.iplot(fig, filename=f, fileopt='overwrite')
plotly_file_box = widgets.Text(value="metacyc_tree", description="Filename")
freetext = widgets.Text()
interact(plotly_graph,G=fixed(DG),df=fixed(metacyc_df), f=plotly_file_box, text=freetext, __manual=True)

### Saving the graph to file

The graph 'G' is saved in [pickle format](https://docs.python.org/3/library/pickle.html), a serialized byte stream of a Python object. This format will preserve Python objects used as nodes or edges.

In [ ]:
def write_graph(f): nx.write_gpickle(DG, f)
graph_sel = widgets.Text(description="Filename", value="datadir/metacyc_tree.gpickle")
interact(write_graph,f=graph_sel, __manual=True)

To read the graph from file simply run: nx.read_gpickle("datadir/metacyc_shell_network.gpickle")